In [110]:
import pandas as pd
import sqlite3

In [111]:
con = sqlite3.connect("northwind.db")
cursor = con.cursor()

table_names = pd.DataFrame(cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"))
table_names

,0
0,Categories
1,sqlite_sequence
2,CustomerCustomerDemo
3,CustomerDemographics
4,Customers
5,Employees
6,EmployeeTerritories
7,Order Details
8,Orders
9,Products


In [112]:
customers = pd.read_sql_query("SELECT * FROM Customers", con)
orders = pd.read_sql_query("SELECT * FROM Orders", con)

In [113]:
customers["City"].unique()

array(['Berlin', 'México D.F.', 'London', 'Luleå', 'Mannheim',
       'Strasbourg', 'Madrid', 'Marseille', 'Tsawassen', 'Buenos Aires',
       'Bern', 'Sao Paulo', 'Aachen', 'Nantes', 'Graz', 'Lille', 'Bräcke',
       'München', 'Torino', 'Lisboa', 'Barcelona', 'Sevilla', 'Campinas',
       'Eugene', 'Caracas', 'Rio de Janeiro', 'San Cristóbal', 'Elgin',
       'Cork', 'Cowes', 'Brandenburg', 'Versailles', 'Toulouse',
       'Vancouver', 'Walla Walla', 'Frankfurt a.M.', 'San Francisco',
       'Barquisimeto', 'I. de Margarita', 'Portland', 'Bergamo',
       'Bruxelles', 'Montréal', 'Leipzig', 'Anchorage', 'Köln', 'Paris',
       'Salzburg', 'Cunewalde', 'Albuquerque', 'Reggio Emilia', 'Genève',
       'Stavern', 'Boise', 'Kobenhavn', 'Lander', 'Charleroi', 'Butte',
       'Münster', 'Kirkland', 'Århus', None, 'Lyon', 'Reims', 'Stuttgart',
       'Oulu', 'Resende', 'Seattle', 'Helsinki', 'Warszawa'], dtype=object)

In [114]:
customers.groupby("City")["CustomerID"].count()

City
Aachen          1
Albuquerque     1
Anchorage       1
Barcelona       1
Barquisimeto    1
               ..
Vancouver       1
Versailles      1
Walla Walla     1
Warszawa        1
Århus           1
Name: CustomerID, Length: 69, dtype: int64

In [115]:
orders.groupby("CustomerID")["OrderID"].count().sort_values(ascending=False).head(5)

CustomerID
BSBEV    210
RICAR    203
LILAS    203
GOURL    202
PRINI    200
Name: OrderID, dtype: int64

In [116]:
order_details = pd.read_sql_query("SELECT * FROM `Order Details`", con)
products = pd.read_sql_query("SELECT * FROM Products", con)

order_details.head()

,OrderID,ProductID,UnitPrice,Quantity,Discount
0,10248,11,14.0,12,0.0
1,10248,42,9.8,10,0.0
2,10248,72,34.8,5,0.0
3,10249,14,18.6,9,0.0
4,10249,51,42.4,40,0.0


In [117]:
joined = order_details.merge(products, on='ProductID', how='left')
joined.head()

,OrderID,ProductID,UnitPrice_x,Quantity,Discount,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice_y,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,10248,11,14.0,12,0.0,Queso Cabrales,5,4,1 kg pkg.,21.00,22,30,30,0
1,10248,42,9.8,10,0.0,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00,26,0,0,1
2,10248,72,34.8,5,0.0,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.80,14,0,0,0
3,10249,14,18.6,9,0.0,Tofu,6,7,40 - 100 g pkgs.,23.25,35,0,0,0
4,10249,51,42.4,40,0.0,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,20,0,10,0


In [126]:
joined.groupby("ProductID")["Quantity"].sum()

ProductID
1     201905
2     201802
3     202186
4     198726
5     199627
       ...  
73    199042
74    202113
75    201021
76    201478
77    201852
Name: Quantity, Length: 77, dtype: int64

In [119]:
rev = (order_details['Quantity'] * order_details['UnitPrice'])
joined['Revenue'] = rev
order_details['Revenue'] = rev
joined[['OrderID', 'Revenue']].sort_values(by='Revenue', ascending=False).head(5)

,OrderID,Revenue
1620,10865,15810.0
1893,10981,15810.0
579922,25805,13175.0
92766,13402,13175.0
143603,14674,13175.0


In [120]:
df = orders.merge(order_details, on='OrderID', how='left')
df.merge(customers, on='CustomerID', how='left')
df.merge(products, on='ProductID', how='left')
df.head()

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,ProductID,UnitPrice,Quantity,Discount,Revenue
0,10248,VINET,5,2016-07-04,2016-08-01,2016-07-16,3,16.75,Vins et alcools Chevalier,59 rue de l-Abbaye,Reims,Western Europe,51100,France,11,14.0,12,0.0,168.0
1,10248,VINET,5,2016-07-04,2016-08-01,2016-07-16,3,16.75,Vins et alcools Chevalier,59 rue de l-Abbaye,Reims,Western Europe,51100,France,42,9.8,10,0.0,98.0
2,10248,VINET,5,2016-07-04,2016-08-01,2016-07-16,3,16.75,Vins et alcools Chevalier,59 rue de l-Abbaye,Reims,Western Europe,51100,France,72,34.8,5,0.0,174.0
3,10249,TOMSP,6,2016-07-05,2016-08-16,2016-07-10,1,22.25,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany,14,18.6,9,0.0,167.4
4,10249,TOMSP,6,2016-07-05,2016-08-16,2016-07-10,1,22.25,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany,51,42.4,40,0.0,1696.0


In [121]:
df.groupby('ShipCountry')['Revenue'].sum()

ShipCountry
Argentina      15542241.62
Austria        10267591.93
Belgium         9182472.91
Brazil         46258164.25
Canada         14641766.32
Denmark        10132404.75
Finland         9817972.82
France         49125550.43
Germany        59109113.22
Ireland         4521586.06
Italy          14443963.20
Mexico         25438604.89
Norway          4599279.63
Poland          5956182.93
Portugal       10122547.39
Spain          19136130.83
Sweden         10445419.72
Switzerland    10275791.38
UK             36259297.32
USA            63874320.00
Venezuela      19324897.12
Name: Revenue, dtype: float64

In [122]:
df.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False).head(5)

CustomerID
BSBEV    6154115.34
HUNGC    5698023.67
RANCH    5559110.08
GOURL    5552597.90
ANATR    5534356.65
Name: Revenue, dtype: float64